In [1]:
%load_ext autoreload
%autoreload 2

import os
import sys
from joblib import dump, load

module_path = os.path.abspath(os.path.join('..', 'code'))
if module_path not in sys.path:
    sys.path.append(module_path)

In [2]:
import utils

## Procesado de datos Oferta de cursos

In [3]:
df = utils.load_data('../data/courses.csv')

In [4]:
def initial_filter(df):
    df = df[df['Lugar']=='CAMPUS PRINCIPAL']
    # Mantener solo los edificos usados para la investigación
    df = df[df['Edificio'].str.contains(r'.*SD.*|.*\.centro.*|.*ML.*|.*LL.*', na=False)]
    # Eliminar filas duplicadas
    df.drop_duplicates(inplace=True)
    
    return df

In [5]:
def further_processing(df):
    # Mantener solo las columnas necesarias
    df = df[['Curso','NRC','Periodo','Edificio','Salón','Días','Horas', 'Inscritos']]
    # Eliminar cursos sin inscritos
    df = df[df['Inscritos']>0]
    return df

In [6]:
def save_file(processed_df, path):
    processed_df.to_csv(path, index=False)

In [7]:
def process_file(input_path, output_path):
    df = utils.load_data(input_path)
    df = initial_filter(df)
    df = further_processing(df)
    save_file(df, output_path)

In [8]:
dump(process_file, '../models/process_file.joblib')

['../models/process_file.joblib']

In [9]:
process_file_loaded = load('../models/process_file.joblib')

process_file_loaded('../data/courses.csv', '../data/processed_courses.csv')



### Usando los datos de talanqueras

In [35]:
import pandas as pd
import numpy as np
import re

In [36]:
courses = pd.read_csv('../data/processed_courses.csv')
talanqueras = pd.read_csv('../data/output.csv')

In [37]:
courses.shape

(4457, 8)

In [38]:
talanqueras['Edificio'] = talanqueras['Puerta'].str.split('-', expand=True)[0]
talanqueras['Edificio'] = talanqueras['Edificio'].str.replace(r'[0-9]', '', regex=True)

courses['Salón'] = courses['Salón'].str.replace('.','')
courses['Edificio'] = courses['Salón'].str.split('_', expand=True)[0]

In [39]:
courses.sample(10)

,Curso,NRC,Periodo,Edificio,Salón,Días,Horas,Inscritos
630,CISO-4506 METODOLOGIA CUALITATIVA PARA LAS CIE...,25131,202420,ML,ML_511,V,0630-0920,22
3480,MATE-1201 PRECÁLCULO,55676,202420,RGD,RGD_07,M,0930-1050,32
1632,GPUB-5601 MODULO INTRODUCTORIO,71193,202420,RGD,RGD_001,L,0630-0820,21
4206,MMBA-4300 MERCADEO ESTRATÉGICO (CICLO 2 DE 8 S...,72888,202420,SD,SD_715,V,1800-2050,36
2506,ISIS-1221 INTRODUCCIÓN A LA PROGRAMACIÓN,41892,202420,SD,SD_303,M,0800-0920,22
2296,IMEC-3709 PROYECTO 3,66407,202420,RGD,RGD_005,V,1100-1220,51
1901,ICYA-4715 MODELACION DE HIDROSISTEMAS,71260,202420,SD,SD_201-2,L,1700-1820,29
3527,MATE-1201 PRECÁLCULO,72803,202420,ML,ML_614,L,1600-1650,32
3168,LENG-1162 ENGLISH 10B: SPEAKING AND WRITING FO...,42324,202420,ML,ML_606,I,1400-1520,14
1414,ESCR-1101 ESCRITURA UNIVERSITARIA I (CICLO 1 D...,51528,202420,SD,SD_302,V,1600-1750,20


In [40]:
def count_entradas(edificio):
    return talanqueras['Edificio'].str.contains(edificio).sum()

In [41]:
edif = pd.DataFrame()
edif['Edificio'] = courses['Edificio'].unique()
for edificio in edif['Edificio']:
    edif.loc[edif['Edificio']==edificio, 'Entradas'] = count_entradas(edificio)
edif.head()  #  Conteo de entradas por edificio

,Edificio,Entradas
0,RGD,10088.0
1,ML,35062.0
2,SD,35198.0
3,LL,10014.0


In [42]:
# df para contar entradas por rangos de horas
hours_count = pd.DataFrame()
hours_count['Edificio'] = courses['Edificio']
hours_count['Rango de Horas'] = talanqueras['FechaHora'].str.split(' ', expand=True)[1].str.split(':', expand=True)[0].astype(float) 

In [43]:
hours_count.head()

,Edificio,Rango de Horas
0,RGD,7.0
1,RGD,6.0
2,ML,7.0
3,ML,12.0
4,RGD,15.0


In [44]:
import numpy as np

def create_ranges(df):
    earliest = float(hours_count['Rango de Horas'].min())
    latest = float(hours_count['Rango de Horas'].max())
    df['Rango de Horas'] = pd.cut(df['Rango de Horas'], bins=np.arange(earliest, latest, 0.5), right=False)
    df.dropna(inplace=True)
    df.drop_duplicates(inplace=True)
    df.reset_index(drop=True, inplace=True)
    return df

create_ranges(hours_count)

,Edificio,Rango de Horas
0,RGD,"[7.0, 7.5)"
1,RGD,"[6.0, 6.5)"
2,ML,"[7.0, 7.5)"
3,ML,"[12.0, 12.5)"
4,RGD,"[15.0, 15.5)"
...,...,...
86,ML,"[4.0, 4.5)"
87,LL,"[2.0, 2.5)"
88,RGD,"[2.0, 2.5)"
89,LL,"[1.0, 1.5)"


In [45]:
#count entries by hour
hours_count['Entradas'] = 0
for i in range(len(hours_count)):
    edificio = hours_count['Edificio'].iloc[i]
    left = hours_count['Rango de Horas'].iloc[i].left
    right = hours_count['Rango de Horas'].iloc[i].right
    for j in range(len(talanqueras)):
        hora = int(talanqueras['FechaHora'].iloc[j].split(' ')[1].split(':')[0])
        building = talanqueras['Edificio'].iloc[j]
        if (hora == left or hora == right) and building == edificio:
            hours_count['Entradas'].iloc[i] += 1

C:\Users\carlo\AppData\Local\Temp\ipykernel_13516\2067999010.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  hours_count['Entradas'].iloc[i] += 1


In [46]:
#Ordenar por edificio y rango de horas
hours_count = hours_count.sort_values(by=["Edificio", "Rango de Horas"])

In [47]:
hours_count

,Edificio,Rango de Horas,Entradas
90,LL,"[0.0, 0.5)",22
89,LL,"[1.0, 1.5)",34
87,LL,"[2.0, 2.5)",27
68,LL,"[3.0, 3.5)",26
83,LL,"[4.0, 4.5)",25
...,...,...,...
35,SD,"[18.0, 18.5)",1043
50,SD,"[19.0, 19.5)",1087
30,SD,"[20.0, 20.5)",957
16,SD,"[21.0, 21.5)",681


In [48]:
# Exportamos los 2 df a csv
edif.to_csv('../data/entradas_edificio.csv', index=False)
hours_count.to_csv('../data/entradas_edificio_hora.csv', index=False)

## Agrupacion de datos usando oferta de cursos

In [20]:
new = courses.copy()

In [21]:
new

,Curso,NRC,Periodo,Edificio,Salón,Días,Horas,Inscritos
0,ADMI-1102 FUNDAMENTOS DE ADMINISTRACION Y GERE...,25454,202420,RGD,RGD_206-7,L,0630-0750,43
1,ADMI-1102 FUNDAMENTOS DE ADMINISTRACION Y GERE...,25454,202420,RGD,RGD_206-7,I,0630-0750,43
2,ADMI-1102 FUNDAMENTOS DE ADMINISTRACION Y GERE...,11411,202420,ML,ML_515,I,0800-0920,46
3,ADMI-1102 FUNDAMENTOS DE ADMINISTRACION Y GERE...,11411,202420,ML,ML_606,L,0800-0920,46
4,ADMI-1102 FUNDAMENTOS DE ADMINISTRACION Y GERE...,11411,202420,RGD,RGD_112-13,I,0800-0920,46
...,...,...,...,...,...,...,...,...
4452,SPUB-4545 FUNDAMENTOS DEL DERECHO A LA SALUD,66159,202420,RGD,RGD_311,V,1400-1650,11
4453,SPUB-4545 FUNDAMENTOS DEL DERECHO A LA SALUD,66159,202420,RGD,RGD_311,V,1400-1650,11
4454,SPUB-4545 FUNDAMENTOS DEL DERECHO A LA SALUD,66159,202420,RGD,RGD_311,V,1400-1650,11
4455,SPUB-4545 FUNDAMENTOS DEL DERECHO A LA SALUD,66159,202420,RGD,RGD_311,V,1400-1650,11


In [22]:
new.drop(columns=['Curso','NRC','Periodo','Salón', 'Días'], inplace=True)

In [23]:
new.sort_values(by=['Edificio','Horas'], inplace=True)
new.drop_duplicates(subset=['Edificio','Horas'], keep='first', inplace=True)
new.reset_index(drop=True, inplace=True)

In [24]:
def convert_to_range(horas):
    # Split the start and end times
    start, end = horas.split("-")

    start_hour = int(start[:2])  # First two digits (hours)
    end_hour = int(end[:2])      # First two digits (hours)
    if end[2:] != '00':
        end_hour += 1

    return[start_hour, end_hour]

# Apply the conversion function
new["Rango de Horas"] = new["Horas"].apply(convert_to_range)

In [25]:
new.drop(columns=['Horas','Rango de Horas'], inplace=True)

In [26]:
#Nos da el numero de estudiantes inscritos en un curso por edificio
new = new.groupby('Edificio').count()*new['Inscritos'].sum()

In [27]:
new

,Inscritos
Edificio,
LL,256168
ML,374880
RGD,331144
SD,374880


In [28]:
new.to_csv('../data/inscritos_edificio.csv')

In [ ]:
import pandas as pd
import numpy as np


df = pd.read_csv('../data/entradas_edificio_hora_completos.csv')

# Convert "Rango de Horas" to numeric ranges for easier processing
def parse_range(range_str):
    range_str = range_str.replace("(", "[").replace(")", "]").strip()
    start, end = map(float, range_str.strip("[]").split(", "))
    return start, end

df[["Start", "End"]] = df["Rango de Horas"].apply(parse_range).apply(pd.Series)

# Initialize an empty DataFrame to store results
all_ranges = []

# Process each unique "Edificio" separately
for edificio in df["Edificio"].unique():
    # Filter data for the current building
    edificio_df = df[df["Edificio"] == edificio]

    # Generate a complete range of intervals for this building
    start_min = edificio_df["Start"].min()
    end_max = edificio_df["End"].max()
    interval = 0.5
    complete_ranges = pd.DataFrame({
        "Start": np.arange(start_min, end_max, interval),
        "End": np.arange(start_min + interval, end_max + interval, interval),
        "Edificio": edificio  # Add the building identifier
    })

    # Merge with the filtered DataFrame
    merged = complete_ranges.merge(edificio_df, on=["Start", "End"], how="left")

    # Drop the redundant "Edificio_x" or "Edificio_y" columns and keep the correct one
    merged["Edificio"] = merged["Edificio_x"].fillna(merged["Edificio_y"])  # Select correct column
    merged.drop(columns=["Edificio_x", "Edificio_y"], inplace=True)  # Drop the redundant columns

    # Fill missing "Entradas" with 0
    merged["Entradas"].fillna(0, inplace=True)

    # Add to the results
    all_ranges.append(merged)

# Combine results from all buildings
final_df = pd.concat(all_ranges, ignore_index=True)

# Recreate "Rango de Horas" column for consistency
final_df["Rango de Horas"] = "[" + final_df["Start"].astype(str) + ", " + final_df["End"].astype(str) + "]"

# Drop temporary columns
final_df = final_df[["Edificio", "Rango de Horas", "Entradas"]]

# Sort and display the result
final_df = final_df.sort_values(by=["Edificio", "Rango de Horas"])

Columns of complete_ranges for LL: ['Start', 'End', 'Edificio']
Columns of merged DataFrame for LL: ['Start', 'End', 'Edificio_x', 'Edificio_y', 'Rango de Horas', 'Entradas']
Columns of complete_ranges for ML: ['Start', 'End', 'Edificio']
Columns of merged DataFrame for ML: ['Start', 'End', 'Edificio_x', 'Edificio_y', 'Rango de Horas', 'Entradas']
Columns of complete_ranges for RGD: ['Start', 'End', 'Edificio']
Columns of merged DataFrame for RGD: ['Start', 'End', 'Edificio_x', 'Edificio_y', 'Rango de Horas', 'Entradas']
Columns of complete_ranges for SD: ['Start', 'End', 'Edificio']
Columns of merged DataFrame for SD: ['Start', 'End', 'Edificio_x', 'Edificio_y', 'Rango de Horas', 'Entradas']
Columns of final_df before checking for 'Edificio': ['Start', 'End', 'Rango de Horas', 'Entradas', 'Edificio']


In [77]:
for edificio in final_df['Edificio'].unique():
    # Filter rows where 'Edificio' matches and 'Entradas' is 0
    condition = (final_df['Edificio'] == edificio) & (final_df['Entradas'] == 0)
    # Compute the mean for the current 'Edificio', ignoring zeros
    mean_value = final_df.loc[final_df['Edificio'] == edificio, 'Entradas'].mean()
    # Assign the mean value to rows where 'Entradas' is 0
    final_df.loc[condition, 'Entradas'] = mean_value


In [78]:
final_df

,Edificio,Rango de Horas,Entradas
0,LL,"[0.0, 0.5]",22.000000
1,LL,"[0.5, 1.0]",217.955556
2,LL,"[1.0, 1.5]",34.000000
3,LL,"[1.5, 2.0]",217.955556
20,LL,"[10.0, 10.5]",669.000000
...,...,...,...
150,SD,"[7.5, 8.0]",766.844444
151,SD,"[8.0, 8.5]",2327.000000
152,SD,"[8.5, 9.0]",766.844444
153,SD,"[9.0, 9.5]",2442.000000


In [79]:
final_df.to_csv('../data/entradas_edificio_hora_completos.csv', index=False)